In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

In [2]:
def modify(e):
    e_split = e.split(':')
    size = len(e_split)
    if size == 2:
        engine = e_split[1][1:]
        return engine
    else:
        return e

In [3]:
def numbers_only_int(s):
    retVal = re.sub("[^0-9]", "", s)
    if retVal == "":
        return 0
    else:
        return int(retVal)

In [4]:
def numbers_only_str(s):
    retVal = re.sub("[^0-9]", "", s)
    if retVal == "":
        return 0
    else:
        return str(retVal)

In [5]:
def categorize_model(s):
    value = 4
    counts = df['Model'].value_counts()
    if counts[s] < 50:
        return 'Other'
    else:
        return s

In [6]:
def categorize_engine(s):
    value = 4
    counts = df['Engine'].value_counts()
    if counts[s] < 100:
        return 'Other'
    else:
        return s

In [7]:
dfs = []
data = 'data/carfax_m'
for i in range(1,20):
    dfs.append(pd.read_csv(data+str(i)+'.csv'))
df = pd.concat(dfs)
df = df[['Year','Brand','Model','Cost','Mileage'
         ,'Accident','Body','Engine','Owners','Usefor','Services']].reset_index()
df = df.applymap(lambda x: str(x).strip())

In [ ]:
df['Model'] = df['Model'].apply(categorize_model)
df['Year'] = df['Year'].apply(lambda x: float(x))
df['Mileage'] = df['Mileage'].apply(lambda x: float(x))
df['Engine'] = df['Engine'].apply(modify)
df['Engine'] = df['Engine'].apply(categorize_engine)

In [ ]:
# np.unique(df['Brand'])
ble = LabelEncoder()
brand_labels = ble.fit_transform(df['Brand'])
brand_mappings = {index: label for index, label in 
                  enumerate(ble.classes_)}
df['Brandl'] = brand_labels

# np.unique(df['Engine'])
ele = LabelEncoder()
engine_labels = ele.fit_transform(df['Engine'])
engine_mappings = {index: label for index, label in 
                  enumerate(ele.classes_)}
df['Enginel'] = engine_labels

# np.unique(df['Body'])
ble = LabelEncoder()
body_labels = ble.fit_transform(df['Body'])
body_mappings = {index: label for index, label in 
                  enumerate(ble.classes_)}
df['Bodyl'] = body_labels

# np.unique(df['Accident'])
ale = LabelEncoder()
acc_labels = ale.fit_transform(df['Accident'])
acc_mappings = {index: label for index, label in 
                  enumerate(ale.classes_)}
df['Accl'] = acc_labels

# np.unique(df['Usefor'])
ule = LabelEncoder()
use_labels = ule.fit_transform(df['Usefor'])
use_mappings = {index: label for index, label in 
                  enumerate(ule.classes_)}
df['Usel'] = use_labels

mle = LabelEncoder()
model_labels = mle.fit_transform(df['Model'])
model_mappings = {index: label for index, label in 
                  enumerate(mle.classes_)}
df['Modell'] = model_labels

In [ ]:
model_mappings

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import scipy.stats as stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

import math

In [ ]:
cdf = df[['Cost','Year','Mileage','Owners','Services','Brandl',
         'Usel','Enginel','Bodyl','Accl','Modell']].copy()
cdf['Owners'] = cdf['Owners'].apply(numbers_only_int)
cdf['Services'] = cdf['Services'].apply(lambda x: int(x))
cdf['Cost'] = cdf['Cost'].apply(numbers_only_int)
cdf = cdf[cdf['Cost'] != 0]

In [ ]:
sns.pairplot(cdf)
plt.savefig('Pairplot_simple.png')

In [ ]:
X, y = cdf.drop('Cost',axis=1), cdf['Cost']
X, X_test, y, y_test = train_test_split(X , y,test_size = .2, random_state=52)
X_train, X_val, y_train, y_val = train_test_split(X , y,test_size = .25, random_state=32)

lr = LinearRegression()
lr.fit(X_train,y_train)
print("X and y train score:",lr.score(X_train,y_train))
print("X and y validation score:",lr.score(X_val,y_val))
print("X and y original test score:",lr.score(X,y))

In [ ]:
preds = lr.predict(X)
sns.jointplot(x=preds,y=y,kind='reg',joint_kws={'line_kws':{'color':'red'}})
plt.savefig('Jointplot_simple.png')

In [ ]:
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(cdf.corr(),cmap='flare',annot=True,vmin=-1,vmax=1)

In [ ]:
ydf = cdf['Year'].apply(lambda x: 2021-x)
# mdf = cdf['Mileage'] / ydf['Year']
cdf['YxM'] = cdf['Mileage'] / ydf['Year']
cdf['YxM'] = cdf['YxM'].apply(lambda x: 0 if math.isinf(x) else x)

In [ ]:
new_df = cdf[['Cost','Year','Mileage','Owners','Services','Brandl',
         'Usel','Enginel','Bodyl','Accl','Modell','YxM']]

X, y = new_df.drop('Cost',axis=1), cdf['Cost']

X, X_test, y, y_test = train_test_split(X , y,test_size = .2, random_state=62)
X_train, X_val, y_train, y_val = train_test_split(X , y,test_size = .25, random_state=22)

lr = LinearRegression()
lr.fit(X_train,y_train)
print("X and y train score:",lr.score(X_train,y_train))
print("X and y validation score:",lr.score(X_val,y_val))
print("X and y original test score:",lr.score(X,y))

In [ ]:
# new_df = pd.get_dummies(cdf, columns = ['Accl'],drop_first=True)
new_df = pd.get_dummies(cdf, columns = ['Brandl'],drop_first=True)
# new_df = pd.get_dummies(cdf, columns = ['Enginel'],drop_first=True)
# new_df = pd.get_dummies(cdf, columns = ['Modell'],drop_first=True)

In [ ]:
X, y = new_df.drop('Cost',axis=1), new_df['Cost']
X, X_test, y, y_test = train_test_split(X , y,test_size = .2, random_state=62)
X_train, X_val, y_train, y_val = train_test_split(X , y,test_size = .25, random_state=22)

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)
print("X and y train score:",lr.score(X_train,y_train))
print("X and y validation score:",lr.score(X_val,y_val))
print("X and y original test score:",lr.score(X,y))

In [ ]:
preds = lr.predict(X)
sns.jointplot(x=preds,y=y,kind='reg',joint_kws={'line_kws':{'color':'red'}})
plt.savefig('Jointplot_better.png')

In [ ]:
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true)) 

In [ ]:
cdf = df[['Cost','Year','Mileage','Owners','Services','Brand',
         'Usel','Enginel','Bodyl','Accl','Modell']].copy()
cdf['Owners'] = cdf['Owners'].apply(numbers_only_int)
cdf['Services'] = cdf['Services'].apply(lambda x: int(x))
cdf['Cost'] = cdf['Cost'].apply(numbers_only_int)
cdf = cdf[cdf['Cost'] != 0]
ydf['Year'] = 2021 - cdf['Year']
cdf['YxM'] = cdf['Mileage'] / ydf['Year']
cdf['YxM'] = cdf['YxM'].apply(lambda x: 0 if math.isinf(x) else x)

In [ ]:
new_df = cdf[['Cost','Year','Mileage','Owners','Services','Brand',
         'Usel','Enginel','Bodyl','Accl','Modell','YxM']]

# new_df['Mileage'] = np.log(new_df['Mileage'])

new_df = pd.get_dummies(cdf, columns = ['Brand'],drop_first=True)

X, y = new_df.drop('Cost',axis=1), cdf['Cost']

X, X_test, y, y_test = train_test_split(X , y,test_size = .2, random_state=62)
X_train, X_val, y_train, y_val = train_test_split(X , y,test_size = .25, random_state=22)

In [ ]:
alphalist = 10**(np.linspace(-2,3,200))
err_vec_val = np.zeros(len(alphalist))
err_vec_train = np.zeros(len(alphalist))

for i,curr_alpha in enumerate(alphalist):

    steps = [('standardize', StandardScaler()), 
             ('lasso', Lasso(alpha = curr_alpha))]

    pipe = Pipeline(steps)
    pipe.fit(X_train, y_train)
    
    val_set_pred = pipe.predict(X_val)
    err_vec_val[i] = mae(y_val, val_set_pred)
np.min(err_vec_val)
alphalist[np.argmin(err_vec_val)]

In [ ]:
plt.plot(np.log10(alphalist), err_vec_val)

In [ ]:
lasso = Lasso(alpha=176)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.fit_transform(X_test.values)

lasso.fit(X_train_scaled,y_train)
coef_l = list(zip(X_train.columns, lasso.coef_))
remove_idx = []
for i in range(len(coef_l)):
    if coef_l[i][1] == 0:
        remove_idx.append(i)
        
print('R^2 score =',lasso.score(X_train_scaled,y_train))
test_set_pred = lasso.predict(X_test_scaled)
print('MAE = ',mae(y_test, test_set_pred))

In [ ]:
sns.jointplot(x=test_set_pred,y=list(y_test),kind='reg',joint_kws={'line_kws':{'color':'red'}})
plt.savefig('lasso.png')

In [ ]:
std = StandardScaler()
std.fit(X_train.values)
X_tr = std.transform(X_train.values)
X_te = std.transform(X_test.values)
alphavec = 10**np.linspace(-2,3,200)

lasso_model = LassoCV(alphas = alphavec, cv=5)
lasso_model.fit(X_tr, y_train)

list(zip(X_train.columns, lasso_model.coef_))
test_set_pred = lasso_model.predict(X_te)

print('R^2 score =',lasso.score(X_tr,y_train))
test_set_pred = lasso_model.predict(X_te)
print('MAE = ',mae(y_test, test_set_pred))

In [ ]:
X_te[0]
list(zip(X_train.columns, lasso_model.coef_))

In [ ]:
X_1 = [[2014,92000,1,6,5,5,6,8,79,13142,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]]
X_te = std.transform(X_1)
print("Toyota 2014 XLE:",lasso_model.predict(X_te))

X_2 = [[2013,111000,5,11,5,5,6,1,59,13875,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
X_te = std.transform(X_2)
print("Acura 2013 Base:",lasso_model.predict(X_te))

X_3 = [[2019,1216,1,1,5,17,9,8,38,26,608,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]]
X_te = std.transform(X_3)
print("Ford 2019 F-150:",lasso_model.predict(X_te))

In [ ]:
lasso_model.predict(X_te)

In [ ]:
model_mappings

In [ ]:
body_mappings

In [ ]:
engine_mappings

In [ ]:
acc_mappings

In [ ]:
brand_mappings

In [ ]:
use_mappings